In [6]:
# pip install -U transformers datasets accelerate evaluate scikit-learn
import pandas as pd
import numpy as np
import evaluate, torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# 1) Load data
df = pd.read_csv("TopGamesDataClean.csv", usecols=["content", "score"])
df["score"] = pd.to_numeric(df["score"], errors="coerce")
df = df[df["score"].between(1,5)].dropna(subset=["content"])
df["content"] = df["content"].astype(str).str.strip()
df = df[df["content"] != ""]

# (Optional) Subsample for speed
df = df.groupby("score", group_keys=False).apply(lambda x: x.sample(n=min(50000, len(x)), random_state=42)).reset_index(drop=True)

# labels must be 0..4 for HF models
label_map = {1:0, 2:1, 3:2, 4:3, 5:4}
df["label"] = df["score"].map(label_map).astype(int)

train_df, test_df = train_test_split(
    df[["content", "label"]],
    test_size=0.2, random_state=42, stratify=df["label"]
)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df.reset_index(drop=True))

# 2) Tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["content"], truncation=True, max_length=256)

train_tok = train_ds.map(tokenize, batched=True, remove_columns=["content"])
test_tok  = test_ds.map(tokenize,  batched=True, remove_columns=["content"])

# 3) Model
num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# 4) Training setup
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
metric_acc = evaluate.load("accuracy")
metric_f1  = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted": metric_f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }

args = TrainingArguments(
    output_dir="distilbert-5cls",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# 5) Evaluate
eval_res = trainer.evaluate()
print(eval_res)

# 6) Predict on test set (labels back to 1..5)
preds = np.argmax(trainer.predict(test_tok).predictions, axis=1)
inv_label_map = {v:k for k,v in label_map.items()}
pred_scores = np.vectorize(inv_label_map.get)(preds)

from sklearn.metrics import classification_report, confusion_matrix
print("\nClassification report (DistilBERT):\n",
      classification_report(np.vectorize(inv_label_map.get)(test_df["label"].values),
                            pred_scores, digits=4))
print("Confusion matrix:\n", confusion_matrix(np.vectorize(inv_label_map.get)(test_df["label"].values), pred_scores))


C:\Users\Austin\AppData\Local\Temp\ipykernel_15260\3285755810.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("score", group_keys=False).apply(lambda x: x.sample(n=min(50000, len(x)), random_state=42)).reset_index(drop=True)


Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Austin\AppData\Local\Temp\ipykernel_15260\3285755810.py:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,1.138300,1.168814,0.503240,0.498717,0.498717
2,1.108300,1.157901,0.510680,0.502485,0.502485
3,1.050900,1.180473,0.509040,0.505794,0.505794


{'eval_loss': 1.1804733276367188, 'eval_accuracy': 0.50904, 'eval_f1_macro': 0.5057936054958516, 'eval_f1_weighted': 0.5057936054958515, 'eval_runtime': 19.6878, 'eval_samples_per_second': 2539.644, 'eval_steps_per_second': 79.389, 'epoch': 3.0}

Classification report (DistilBERT):
               precision    recall  f1-score   support

           1     0.5957    0.5900    0.5928     10000
           2     0.4180    0.4254    0.4217     10000
           3     0.3928    0.3731    0.3827     10000
           4     0.4818    0.4343    0.4568     10000
           5     0.6334    0.7224    0.6750     10000

    accuracy                         0.5090     50000
   macro avg     0.5043    0.5090    0.5058     50000
weighted avg     0.5043    0.5090    0.5058     50000

Confusion matrix:
 [[5900 2548  939  246  367]
 [2335 4254 2175  770  466]
 [1036 2404 3731 1950  879]
 [ 383  764 2040 4343 2470]
 [ 250  207  613 1706 7224]]


In [7]:
# pip install -U transformers datasets accelerate evaluate scikit-learn
import pandas as pd
import numpy as np
import evaluate, torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# 1) Load data
df = pd.read_csv("TopGamesDataClean.csv", usecols=["content", "score"])
df["score"] = pd.to_numeric(df["score"], errors="coerce")
df = df[df["score"].between(1,5)].dropna(subset=["content"])
df["content"] = df["content"].astype(str).str.strip()
df = df[df["content"] != ""]

# (Optional) Subsample for speed
# df = df.groupby("score", group_keys=False).apply(lambda x: x.sample(n=min(50000, len(x)), random_state=42)).reset_index(drop=True)

# labels must be 0..4 for HF models
label_map = {1:0, 2:1, 3:2, 4:3, 5:4}
df["label"] = df["score"].map(label_map).astype(int)

train_df, test_df = train_test_split(
    df[["content", "label"]],
    test_size=0.2, random_state=42, stratify=df["label"]
)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df.reset_index(drop=True))

# 2) Tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["content"], truncation=True, max_length=256)

train_tok = train_ds.map(tokenize, batched=True, remove_columns=["content"])
test_tok  = test_ds.map(tokenize,  batched=True, remove_columns=["content"])

# 3) Model
num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# 4) Training setup
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
metric_acc = evaluate.load("accuracy")
metric_f1  = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted": metric_f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }

args = TrainingArguments(
    output_dir="distilbert-5cls",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# 5) Evaluate
eval_res = trainer.evaluate()
print(eval_res)

# 6) Predict on test set (labels back to 1..5)
preds = np.argmax(trainer.predict(test_tok).predictions, axis=1)
inv_label_map = {v:k for k,v in label_map.items()}
pred_scores = np.vectorize(inv_label_map.get)(preds)

from sklearn.metrics import classification_report, confusion_matrix
print("\nClassification report (DistilBERT):\n",
      classification_report(np.vectorize(inv_label_map.get)(test_df["label"].values),
                            pred_scores, digits=4))
print("Confusion matrix:\n", confusion_matrix(np.vectorize(inv_label_map.get)(test_df["label"].values), pred_scores))


Map:   0%|          | 0/1620599 [00:00<?, ? examples/s]

Map:   0%|          | 0/405150 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Austin\AppData\Local\Temp\ipykernel_15260\2407354302.py:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,0.847700,0.847515,0.690295,0.468372,0.647658
2,0.796200,0.843503,0.691351,0.484785,0.660098
3,0.782500,0.855507,0.690406,0.491885,0.661771


{'eval_loss': 0.8555066585540771, 'eval_accuracy': 0.690406022460817, 'eval_f1_macro': 0.49188483010881556, 'eval_f1_weighted': 0.6617705737910411, 'eval_runtime': 161.9609, 'eval_samples_per_second': 2501.53, 'eval_steps_per_second': 78.173, 'epoch': 3.0}

Classification report (DistilBERT):
               precision    recall  f1-score   support

           1     0.6785    0.8321    0.7475     93654
           2     0.4511    0.1036    0.1685     27683
           3     0.3499    0.2924    0.3186     37901
           4     0.4654    0.3266    0.3839     55666
           5     0.7958    0.8918    0.8411    190246

    accuracy                         0.6904    405150
   macro avg     0.5481    0.4893    0.4919    405150
weighted avg     0.6580    0.6904    0.6618    405150

Confusion matrix:
 [[ 77929   1671   5568   1756   6730]
 [ 14762   2867   5222   2001   2831]
 [ 10555   1302  11082   6860   8102]
 [  5006    358   6245  18183  25874]
 [  6610    158   3554  10267 169657]]


In [9]:
# pip install -U transformers datasets accelerate evaluate scikit-learn
import pandas as pd
import numpy as np
import evaluate, torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# 1) Load data
df = pd.read_csv("TopGamesDataClean.csv", usecols=["content", "score", 'game_name'])
df["score"] = pd.to_numeric(df["score"], errors="coerce")
df = df[df["score"].between(1,5)].dropna(subset=["content"])
df["content"] = df["content"].astype(str).str.strip()
df = df[df["content"] != ""]
df = df[df["game_name"] == "Among Us"]

# (Optional) Subsample for speed
# df = df.groupby("score", group_keys=False).apply(lambda x: x.sample(n=min(50000, len(x)), random_state=42)).reset_index(drop=True)

# labels must be 0..4 for HF models
label_map = {1:0, 2:1, 3:2, 4:3, 5:4}
df["label"] = df["score"].map(label_map).astype(int)

train_df, test_df = train_test_split(
    df[["content", "label"]],
    test_size=0.2, random_state=42, stratify=df["label"]
)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
test_ds  = Dataset.from_pandas(test_df.reset_index(drop=True))

# 2) Tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["content"], truncation=True, max_length=256)

train_tok = train_ds.map(tokenize, batched=True, remove_columns=["content"])
test_tok  = test_ds.map(tokenize,  batched=True, remove_columns=["content"])

# 3) Model
num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# 4) Training setup
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
metric_acc = evaluate.load("accuracy")
metric_f1  = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted": metric_f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }

args = TrainingArguments(
    output_dir="distilbert-5cls",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# 5) Evaluate
eval_res = trainer.evaluate()
print(eval_res)

# 6) Predict on test set (labels back to 1..5)
preds = np.argmax(trainer.predict(test_tok).predictions, axis=1)
inv_label_map = {v:k for k,v in label_map.items()}
pred_scores = np.vectorize(inv_label_map.get)(preds)

from sklearn.metrics import classification_report, confusion_matrix
print("\nClassification report (DistilBERT):\n",
      classification_report(np.vectorize(inv_label_map.get)(test_df["label"].values),
                            pred_scores, digits=4))
print("Confusion matrix:\n", confusion_matrix(np.vectorize(inv_label_map.get)(test_df["label"].values), pred_scores))


Map:   0%|          | 0/57487 [00:00<?, ? examples/s]

Map:   0%|          | 0/14372 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Austin\AppData\Local\Temp\ipykernel_15260\1402960713.py:80: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,1.087700,1.058318,0.556986,0.487351,0.528486
2,0.979900,1.046454,0.567492,0.509907,0.548277
3,0.894500,1.074016,0.565057,0.518349,0.554908


{'eval_loss': 1.0740162134170532, 'eval_accuracy': 0.5650570553854718, 'eval_f1_macro': 0.5183490056698038, 'eval_f1_weighted': 0.5549080762312745, 'eval_runtime': 5.8916, 'eval_samples_per_second': 2439.407, 'eval_steps_per_second': 76.38, 'epoch': 3.0}

Classification report (DistilBERT):
               precision    recall  f1-score   support

           1     0.6332    0.7562    0.6893      3331
           2     0.4129    0.1834    0.2540      1821
           3     0.3940    0.4520    0.4210      2427
           4     0.5046    0.5525    0.5275      3247
           5     0.7323    0.6703    0.6999      3546

    accuracy                         0.5651     14372
   macro avg     0.5354    0.5229    0.5183     14372
weighted avg     0.5603    0.5651    0.5549     14372

Confusion matrix:
 [[2519  229  395  145   43]
 [ 757  334  505  190   35]
 [ 444  165 1097  588  133]
 [ 152   66  577 1794  658]
 [ 106   15  210  838 2377]]


In [ ]:
# 1) Load data
df = pd.read_csv("TopGamesDataClean.csv", usecols=["content", "score", 'game_name'])
df["score"] = pd.to_numeric(df["score"], errors="coerce")
df = df[df["score"].between(1,5)].dropna(subset=["content"])
df["content"] = df["content"].astype(str).str.strip()
df = df[df["content"] != ""]

C:\Users\Austin\AppData\Local\Temp\ipykernel_15260\2380956854.py:105: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,1.196600,1.165655,0.513707,0.422588,0.473560
2,1.114100,1.119280,0.533120,0.448753,0.495597
3,1.029200,1.104034,0.541748,0.482531,0.520431
4,0.996200,1.096651,0.542861,0.494399,0.529183
5,0.961800,1.106020,0.544949,0.496125,0.531706
6,0.899300,1.113924,0.544322,0.493440,0.529268


{'eval_loss': 1.1060199737548828, 'eval_accuracy': 0.5449485109935986, 'eval_f1_macro': 0.4961245010818229, 'eval_f1_weighted': 0.5317055859078008, 'eval_runtime': 1.5915, 'eval_samples_per_second': 9030.576, 'eval_steps_per_second': 71.003, 'epoch': 6.0}

Classification report (BiLSTM):
               precision    recall  f1-score   support

           1     0.6070    0.7550    0.6730      3331
           2     0.3837    0.1840    0.2487      1821
           3     0.3856    0.3869    0.3863      2427
           4     0.4891    0.5057    0.4973      3247
           5     0.6737    0.6771    0.6754      3546

    accuracy                         0.5449     14372
   macro avg     0.5078    0.5017    0.4961     14372
weighted avg     0.5312    0.5449    0.5317     14372

Confusion matrix:
 [[2515  239  351  145   81]
 [ 785  335  429  193   79]
 [ 499  204  939  586  199]
 [ 217   69  515 1642  804]
 [ 127   26  201  791 2401]]


In [16]:
# ---- CONTINUES FROM YOUR df ----
# df already cleaned and contains columns: content (str), score (1..5)

# 1) Load data
df = pd.read_csv("TopGamesDataClean.csv", usecols=["content", "score", 'game_name'])
df["score"] = pd.to_numeric(df["score"], errors="coerce")
df = df[df["score"].between(1,5)].dropna(subset=["content"])
df["content"] = df["content"].astype(str).str.strip()
df = df[df["content"] != ""]

import numpy as np, torch, evaluate
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, TrainingArguments, Trainer
)
from transformers.trainer_utils import IntervalStrategy
from transformers import EarlyStoppingCallback

# 0) Labels 1..5 -> 0..4
label_map = {1:0, 2:1, 3:2, 4:3, 5:4}
df = df[df["score"].between(1,5)]
df["label"] = df["score"].map(label_map).astype(int)

# 1) Tokenizer
model_name = "distilbert-base-uncased"
num_labels = 5
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["content"], truncation=True, max_length=256)

# === SOLUSI 1: class_encode_column + stratified split, lalu tokenisasi ===
# a) Buat Dataset & ubah 'label' menjadi ClassLabel
full_ds = Dataset.from_pandas(df[["content","label"]].reset_index(drop=True))
full_ds = full_ds.class_encode_column("label")  # penting untuk stratify_by_column

# b) Split dengan stratifikasi
splits    = full_ds.train_test_split(test_size=0.1, stratify_by_column="label", seed=42)
train_all = splits["train"]
val_small = splits["test"]

# c) Tokenisasi SETELAH split
train_all = train_all.map(tokenize, batched=True, remove_columns=["content"])
val_small = val_small.map(tokenize,  batched=True, remove_columns=["content"])

# d) (Untuk pelatihan akhir 100% data) tokenisasi seluruh dataset juga
full_tok = full_ds.map(tokenize, batched=True, remove_columns=["content"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
metric_acc = evaluate.load("accuracy")
metric_f1  = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted": metric_f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }

# 2) Hyperparameter tuning (Optuna backend)
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

def compute_objective(metrics):
    return metrics["eval_f1_macro"]

def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-6, 5e-5, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.2),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.0, 0.2),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "per_device_eval_batch_size": trial.suggest_categorical("per_device_eval_batch_size", [16, 32, 64]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 5),
        "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type", ["linear", "cosine", "cosine_with_restarts"]),
    }

base_args = TrainingArguments(
    output_dir="distilbert-5cls",
    eval_strategy=IntervalStrategy.EPOCH,
    save_strategy=IntervalStrategy.EPOCH,
    logging_strategy=IntervalStrategy.STEPS,
    logging_steps=100,
    learning_rate=2e-5,                 # overridden by search
    per_device_train_batch_size=16,     # overridden
    per_device_eval_batch_size=32,      # overridden
    num_train_epochs=3,                 # overridden
    weight_decay=0.01,                  # overridden
    warmup_ratio=0.0,                   # overridden
    lr_scheduler_type="linear",         # overridden
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

trainer = Trainer(
    model_init=model_init,
    args=base_args,
    train_dataset=train_all,
    eval_dataset=val_small,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.0)],
)

best = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=compute_objective,
    n_trials=15,
)
print("Best trial:", best)

# 3) Retrain on 100% of the data with best HPs
best_args = {**base_args.to_dict(), **best.hyperparameters}
final_args = TrainingArguments(
    **best_args,
    output_dir="distilbert-5cls-final",
    load_best_model_at_end=False,      # training on all data; no holdout to pick "best"
    eval_strategy=IntervalStrategy.NO, # disable eval during training
    save_strategy=IntervalStrategy.EPOCH,
)

final_trainer = Trainer(
    model_init=model_init,
    args=final_args,
    train_dataset=full_tok,     # ALL data used here
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

final_trainer.train()

# 4) Optional: quick reference eval on the small split (not a true test set)
ref_eval = Trainer(
    model=final_trainer.model,
    args=final_args,
    eval_dataset=val_small,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
).evaluate()
print("Reference eval on small holdout:", ref_eval)

# 5) Save
final_trainer.model.save_pretrained("distilbert-5cls-final/model")
tokenizer.save_pretrained("distilbert-5cls-final/model")


Stringifying the column:   0%|          | 0/2025749 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/2025749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1823174 [00:00<?, ? examples/s]

Map:   0%|          | 0/202575 [00:00<?, ? examples/s]

Map:   0%|          | 0/2025749 [00:00<?, ? examples/s]

...

[I 2025-11-11 03:25:43,294] Trial 13 pruned. 
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,0.827300,0.848360,0.689636,0.480726,0.654070
2,0.826400,0.841597,0.691048,0.480021,0.657045
3,0.772100,0.841950,0.692193,0.485454,0.658863


[I 2025-11-11 06:26:36,454] Trial 14 finished with value: 0.4854538665412809 and parameters: {'learning_rate': 1.2638607703581072e-05, 'weight_decay': 0.061215082657034826, 'warmup_ratio': 0.007422601170802312, 'per_device_train_batch_size': 32, 'per_device_eval_batch_size': 32, 'num_train_epochs': 3, 'lr_scheduler_type': 'linear'}. Best is trial 5 with value: 0.4903625701541766.


Best trial: BestRun(run_id='5', objective=0.4903625701541766, hyperparameters={'learning_rate': 1.2045308108537165e-05, 'weight_decay': 0.09092300989431466, 'warmup_ratio': 0.17294505407027128, 'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 64, 'num_train_epochs': 4, 'lr_scheduler_type': 'cosine_with_restarts'}, run_summary=None)


TypeError: transformers.training_args.TrainingArguments() got multiple values for keyword argument 'output_dir'

In [17]:
final_trainer.model.save_pretrained("distilbert-5cls-final/model")
tokenizer.save_pretrained("distilbert-5cls-final/model")

('distilbert-5cls-final/model\\tokenizer_config.json',
 'distilbert-5cls-final/model\\special_tokens_map.json',
 'distilbert-5cls-final/model\\vocab.txt',
 'distilbert-5cls-final/model\\added_tokens.json',
 'distilbert-5cls-final/model\\tokenizer.json')

In [18]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer
import numpy as np, evaluate

model_path = "distilbert-5cls-final/model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [19]:
metric_acc = evaluate.load("accuracy")
metric_f1  = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted": metric_f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }


In [20]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

ref_trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    eval_dataset=val_small,  # this was your 10% holdout
)

results = ref_trainer.evaluate()
print("Final model performance:")
for k, v in results.items():
    print(f"{k:<20} : {v:.4f}")


C:\Users\Austin\AppData\Local\Temp\ipykernel_15260\948968231.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  ref_trainer = Trainer(


Final model performance:
eval_loss            : 1.0687
eval_model_preparation_time : 0.0010
eval_accuracy        : 0.6273
eval_f1_macro        : 0.4519
eval_f1_weighted     : 0.6153
eval_runtime         : 151.1461
eval_samples_per_second : 1340.2590
eval_steps_per_second : 167.5330


In [ ]:
import pandas as pd, numpy as np, torch, evaluate
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, TrainingArguments, Trainer,
    EarlyStoppingCallback
)
from transformers.trainer_utils import IntervalStrategy

# ==== 1. Load & clean ====
df = pd.read_csv("TopGamesDataClean.csv", usecols=["content", "score", "game_name"])
df["score"] = pd.to_numeric(df["score"], errors="coerce")
df = df[df["score"].between(1,5)].dropna(subset=["content"])
df["content"] = df["content"].astype(str).str.strip()
df = df[df["content"] != ""]

# 1–5 → 0–4
label_map = {1:0, 2:1, 3:2, 4:3, 5:4}
df["label"] = df["score"].map(label_map).astype(int)

# ==== 2. Model / tokenizer ====
model_name = "microsoft/deberta-v3-base"
num_labels = 5
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["content"], truncation=True, max_length=256)

# ==== 3. Dataset + stratified split ====
full_ds = Dataset.from_pandas(df[["content","label"]].reset_index(drop=True))
full_ds = full_ds.class_encode_column("label")
splits    = full_ds.train_test_split(test_size=0.1, stratify_by_column="label", seed=42)
train_all = splits["train"]
val_small = splits["test"]

train_tok = train_all.map(tokenize, batched=True, remove_columns=["content"])
val_tok   = val_small.map(tokenize, batched=True, remove_columns=["content"])
full_tok  = full_ds.map(tokenize, batched=True, remove_columns=["content"])

# ==== 4. Data collator & metrics ====
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
metric_acc = evaluate.load("accuracy")
metric_f1  = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted": metric_f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }

# ==== 5. Training arguments ====
args = TrainingArguments(
    output_dir="deberta-5cls",
    eval_strategy=IntervalStrategy.EPOCH,
    save_strategy=IntervalStrategy.EPOCH,
    logging_strategy=IntervalStrategy.STEPS,
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    warmup_ratio=0.06,
    lr_scheduler_type="linear",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

# ==== 6. Trainer ====
trainer = Trainer(
    model_init=lambda: AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels),
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

# ==== 7. Evaluate on validation set ====
eval_results = trainer.evaluate()
print("\nValidation results:")
for k, v in eval_results.items():
    if isinstance(v, float):
        print(f"{k:<20} : {v:.4f}")

# ==== 8. Retrain on full data (optional final model) ====
final_args = TrainingArguments(
    output_dir="deberta-5cls-final",
    per_device_train_batch_size=16,
    num_train_epochs=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.06,
    lr_scheduler_type="linear",
    fp16=torch.cuda.is_available(),
    evaluation_strategy=IntervalStrategy.NO,
    save_strategy=IntervalStrategy.EPOCH,
    report_to="none",
)

final_trainer = Trainer(
    model_init=lambda: AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels),
    args=final_args,
    train_dataset=full_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

final_trainer.train()

# ==== 9. Save model ====
final_trainer.model.save_pretrained("deberta-5cls-final/model")
tokenizer.save_pretrained("deberta-5cls-final/model")
print("✅ Model saved at deberta-5cls-final/model")

c:\Users\Austin\anaconda3\envs\Torch\lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Stringifying the column:   0%|          | 0/2025749 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/2025749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1823174 [00:00<?, ? examples/s]

Map:   0%|          | 0/202575 [00:00<?, ? examples/s]

Map:   0%|          | 0/2025749 [00:00<?, ? examples/s]

C:\Users\Austin\AppData\Local\Temp\ipykernel_25496\3135112512.py:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifi

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
1,0.827300,0.844383,0.692168,0.470902,0.646640
2,0.829500,0.839747,0.693486,0.492865,0.666408
3,0.786000,0.841420,0.694572,0.499555,0.667211
